In [6]:
import numpy as np

# Import Sionna
try:
    import sionna
except ImportError as e:
    # Install Sionna if package is not already installed
    import os
    os.system("pip install sionna")
    import sionna
import tensorflow as tf

from utils import *
import matplotlib.pyplot as plt
from termcolor import colored
from scipy.special import erfc

In [7]:
tx_params = {
    "nUsers" : 3,
    "powerAllocation" : np.zeros(3),
    "nCode" : 120,
    "kCode" : 60,
    "codingRate": 1/2,
}


In [8]:
nmodbits = 2
binary_source = sionna.utils.BinarySource()
constellation = sionna.mapping.Constellation("qam", nmodbits)
encoder = sionna.fec.conv.ConvEncoder(rate=tx_params["codingRate"])
mapper = sionna.mapping.Mapper(constellation=constellation)
channel = sionna.channel.AWGN()
demapper = sionna.mapping.Demapper("app","qam", nmodbits, hard_out=True,  dtype=tf.complex128)
decoder = sionna.fec.conv.ViterbiDecoder(encoder=encoder, rate=tx_params["codingRate"])

In [12]:

ebn0 = np.arange(0, 21, 1) #dB
snr_db = ebn0 + 10*np.log10(tx_params["codingRate"]) + 10*np.log10(nmodbits) #dB
nb_seq =1000
alpha = 0

BER_coded = np.zeros(len(ebn0))
BER_uncoded = np.zeros(len(ebn0))

gn = np.array([1, 1])
gd = np.array([1, 1, 1])
R1,R0,out_R1,out_R0 = poly2trellis(gn,gd)
d = 1
out_R1_tf = tf.convert_to_tensor(out_R1)
out_R0_tf = tf.convert_to_tensor(out_R0)
symb_R1 = mapper(out_R1_tf).numpy()
symb_R0 = mapper(out_R0_tf).numpy()

for ii in range(len(snr_db)) :

  BER_coded_ii = np.zeros(nb_seq)
  BER_uncoded_ii = np.zeros(nb_seq)

  var_noise = 1/(10**(0.1*snr_db[ii]))
  len_b = tx_params["kCode"]

  for jj in range(nb_seq) :

    # NOMA Power allocation
    Ptot_db = 15 # total power allocation in dB
    Ptot = 10**(0.1*Ptot_db)
    #polynomial = np.array([1, 3, 3, -1*Ptot/var_noise])
    polynomial = np.array([1, 2, 1, -1*Ptot])
    roots = np.roots(polynomial) # roots of polynomial
    x = roots[np.isreal(roots)][-1] # keep only real root

    #Evaluate the root 
    #print(polynomial[0]*x**3 + polynomial[1]*x**2 + polynomial[2]*x + polynomial[3]	)

    if(len(roots[np.isreal(roots)])>1) : 
      print("More than 2 roots found in polynomial")

    # P1 = x
    # P2 = ((x**2) + x)
    # P3 = ((x**3) + 2*(x**2) + x)

    # P1 = 10*(var_noise)
    # P2 = 10*(P1 + var_noise)
    # P3 = 10*(P1 + P2 + var_noise)

    # factor = Ptot/(P1+P2+P3)
    # P1 *= factor
    # P2 *= factor
    # P3 *= factor

    # P3 = x**3 + x**2
    # P2 = x**2 
    # P1 = x

    mu = (1 + (Ptot)/var_noise)**(-1/tx_params["nUsers"])
    alpha1 = ((mu**(1-1))*(1-mu))/(1-mu**(tx_params["nUsers"]))
    alpha2 = ((mu**(2-1))*(1-mu))/(1-mu**(tx_params["nUsers"]))
    alpha3 = ((mu**(3-1))*(1-mu))/(1-mu**(tx_params["nUsers"]))
    P1 = alpha3*Ptot
    P2 = alpha2*Ptot
    P3 = alpha1*Ptot

    tx_params["powerAllocation"] = np.array([P3, P2, P1])
    if jj==1 : 
    #  fig = plt.figure()
    #  x = np.linspace(-10, 10, 100)
    #  plt.plot(x, polynomial[0] + polynomial[1]*x + polynomial[2]*(x**2) + polynomial[3]*(x**3))
    #  plt.grid()
    #  plt.show()
     print("Noise variance: ", var_noise)
     #print(roots)
     print("P1: ", P1, "P2: ", P2, "P3: ", P3)
     print("Natural scale: ", P1/var_noise, P2/(P1 + var_noise), P3/(P2 + P1 + var_noise))
     print("Log scale: ", 10*np.log10(P1/var_noise), 10*np.log10(P2/(P1 + var_noise)), 10*np.log10(P3/(P2 + P1 + var_noise)))
     print("Sum of powers: ", P1+P2+P3)


    data = np.zeros((tx_params["nUsers"], tx_params["nCode"])) # nCode noramlement (à changer après)
    b = binary_source([tx_params["nUsers"], tx_params["kCode"]])
    #c = encoder(b)
    # encoding
    b_array = b.numpy()
    c_array = np.zeros((tx_params["nUsers"], 2*len(b_array[0])))
    for idx_user in range(tx_params["nUsers"]):
      (u_s,u_c) = conv_encoder(b_array[idx_user],R1,R0,out_R1,out_R0,len_b)
      c_array[idx_user] = np.reshape(np.transpose([u_s,u_c]),(2*len(u_s),))
    c = tf.convert_to_tensor(c_array)
    #c=b 
    x = mapper(c)
    data = x.numpy()

    data_out = NOMA(data, tx_params)
    
    h = Rayleigh_canal(alpha, len(data_out))
    n = (1j*np.random.normal(scale=np.sqrt(var_noise/2), size=data_out.shape)+np.random.normal(scale=np.sqrt(var_noise/2),  size=data_out.shape))

    #y_canal = h*data_out+n
    #y_canal = channel([data_out, var_noise])
    y_canal = data_out + n

    # Reception 
    #y_canal = y_canal.reshape(1, len(y_canal))
    #y_received = y_canal/h
    y_received = y_canal
    y_decoded = np.zeros((tx_params["nUsers"], tx_params["kCode"]))
    y_demapped = np.zeros((tx_params["nUsers"], tx_params["nCode"])) # nCode noramlement (à changer après)
    # SIC 
    for idx_user in range(tx_params["nUsers"]):

      y = demapper([tf.convert_to_tensor(y_received), var_noise])

      y_demapped[idx_user, :] = y.numpy()

      #y_user = decoder(tf.convert_to_tensor(y.numpy().reshape((1, tx_params["nCode"]))))
      y_user = viterbi_decoder(R1,R0,symb_R1, symb_R0,len_b, y.numpy())
      #y_user = tf.convert_to_tensor(y.numpy().reshape((1, len(y))))

      y_decoded[idx_user, :] = y_user#.numpy()

      #y_user_encoded = encoder(y_user)
      (u_s,u_c) = conv_encoder(y_user,R1,R0,out_R1,out_R0,len_b)
      y_user_encoded = np.reshape(np.transpose([u_s,u_c]),(2*len(u_s),))
      y_user_encoded = tf.convert_to_tensor(y_user_encoded.reshape((1, len(y_user_encoded))))
    
      y_received = y_received - mapper(y_user_encoded).numpy()[0]*np.sqrt(tx_params["powerAllocation"][idx_user])

    y_demapped_tf = tf.convert_to_tensor(y_demapped)
    y_decoded_tf = tf.convert_to_tensor(y_decoded)
    c_hat = tf.cast(tf.less(0.0, y_demapped_tf), tf.float32)
    b_hat = tf.cast(tf.less(0.0, y_decoded_tf), tf.float32)
    ber_uncoded = sionna.utils.metrics.compute_ber(c, c_hat)
    ber_coded = sionna.utils.metrics.compute_ber(b, b_hat)

    BER_coded_ii[jj] = ber_coded.numpy()
    BER_uncoded_ii[jj] = ber_uncoded.numpy()
  
  BER_coded[ii] = np.mean(BER_coded_ii)
  BER_uncoded[ii] = np.mean(BER_uncoded_ii)

  print("Eb/N0 = " + str(ebn0[ii]) +  "dB, BER uncoded = " +  str(BER_uncoded[ii]) + ", BER coded = " + str(BER_coded[ii]))
  

data_out:  (60,)


ValueError: could not broadcast input array from shape (120,) into shape (60,)

In [ ]:
# Plot

plt.figure()
plt.semilogy(snr_db, BER_coded, "v-", label="coded", color="black")
plt.semilogy(snr_db, BER_uncoded, "D-", label="NOMA (with fading)", color="black")
ber = (1/2)*erfc(np.sqrt(10**(0.1*snr_db)/2))
plt.plot(snr_db, ber, label="AWGN", color="black")
plt.xlabel("SNR [dB]")
plt.ylabel("BER")
plt.xscale("linear")
plt.yscale("log")
plt.grid(which="both")
plt.legend()
#plt.xlim([0, 15])
plt.ylim([10**(-8), 0.5])
plt.show()